In [14]:
class GameState:
    len = 0
    width = 0
    
    currentStateValues = []
  #  choosing option between cloning the values or extracting them from txt file
    def __init__(self, arg):
        if isinstance(arg, str):
            self.__init_from_file(arg)
        else:
            self.__init_with_cloning(arg)
            
 # extracting the string from txt file and converting it to int       
    def __init_from_file(self, path: str):
        self.currentStateValues=[]
        with open(path) as gamefile:
            firstline = gamefile.readline()
            dim = firstline.split(",", 2)
            self.width = int(dim[0])
            self.len = int(dim[1])
            for line in gamefile:
                rowData = line.split(",", self.width)
                rowDataInt = []
                for i in range(self.width):
                    rowDataInt.append(int(rowData[i]))
                self.currentStateValues.append(rowDataInt)
                
  #function to clone the state  
    def __init_with_cloning(self, gs):
        self.currentStateValues = []
        self.len = gs.len
        self.width = gs.width
        for i in range(self.len):
            row = []
            for j in range(self.width):
                row.append(gs.currentStateValues[i][j])
            self.currentStateValues.append(row)
            
 #performing normalization to transform state to normal form               
    def normalization(self):
        nextIdx = 3;
        for x in range(self.len):
            for y in range(self.width):
                if self.currentStateValues[x][y]==nextIdx:
                    nextIdx=nextIdx+1
                elif self.currentStateValues[x][y]>nextIdx:
                    self.swapIdx(nextIdx,self.currentStateValues[x][y])
                    nextIdx=nextIdx+1
                    
#performing swap for above values
    def swapIdx(self,idx1,idx2):
        for i in range(self.len):
            for j in range(self.width):
                if self.currentStateValues[i][j]==idx1:
                    self.currentStateValues[i][j]=idx2
                elif self.currentStateValues[i][j]==idx2:
                    self.currentStateValues[i][j]==idx1
  
 #printing the current state
    def printCurrentState(self):
        if not self.currentStateValues or len(self.currentStateValues) == 0:
            print("GameState is empty")
        else:
            for row in self.currentStateValues:
                print(row)

 #check if the master brick has reached the goal               
    def checkresult(self):
        result=1
        for y in range(self.width):
            if self.currentStateValues[0][y]==-1 and self.currentStateValues[1][y]==2:
                result=0
                break
            elif self.currentStateValues[self.len-1][y]==-1 and self.currentStateValues[self.len-2][y]==2:
                result=0 
                break
        for x in range(self.len):   
            if self.currentStateValues[x][0]==-1 and self.currentStateValues[x][1]==2:
                result=0
                break
            elif self.currentStateValues[x][self.width-1]==-1 and self.currentStateValues[x][self.width-2]==2:
                result=0
                break
        if result == 0:
            print("Goal reached")
            self.printCurrentState()
        return result

#comparing two states    
    def stateCompare(self, otherGS):
        self.normalization()
        otherGS.normalization()
        return self.currentStateValues == otherGS.currentStateValues


        
class PossibleMove:
    xcord=0
    ycord=0
    dirc=""
    shp=""
    gs=None
    nextPossibleMoves=[]
    
    def __init__(self):
        self.gs=None
        self.nextPossibleMoves=[]
        
 #checking if the node( state here) already exist      
    
    def checkIfGameStateAlreadyExists(self, otherGS):
        if (otherGS.stateCompare(self.gs)):
            return 1
        for move in self.nextPossibleMoves:
            if move.checkIfGameStateAlreadyExists(otherGS):
                return 1
        return 0

    
class MoveBlock: 
    
    #finding shape of each block and calling findDirection method        
    def selectBlock(self,gs:GameState):
        allpossmoves=[]
        for m in range(gs.len):
            for n in range(gs.width):
                if gs.currentStateValues[m][n]>1:
                    if gs.currentStateValues[m][n]==gs.currentStateValues[m][n+1]:
                        shape="wide"
                        d=self.findDirection(m,n,shape,gs)
                        allpossmoves.extend(d)
                    elif gs.currentStateValues[m][n]==gs.currentStateValues[m+1][n]:
                        shape="long"
                        d=self.findDirection(m,n,shape,gs)
                        allpossmoves.extend(d)
                        
                     
                    elif gs.currentStateValues[m][n]!=gs.currentStateValues[m-1][n] and gs.currentStateValues[m][n]!=gs.currentStateValues[m][n-1] and gs.currentStateValues[m][n]!=gs.currentStateValues[m+1][n] and gs.currentStateValues[m][n]!=gs.currentStateValues[m][n+1]:
                        shape="smallsquare"
                        #length=length+1
                        d=self.findDirection(m,n,shape,gs)
                        allpossmoves.extend(d)
                        #print("block name, shape and directions   ",gs.currentStateValues[m][n],shape,d)
        return allpossmoves
                            #print(m,n,shape)    
#finding all possible moves for all kinds of blocks
    def findDirection(self,m,n,shape,gs):
        direction=[]
        possmovers=[]
        if shape=="wide":
            f=self.boundryCheck(m,n-1,gs)
            #checking if boundry conditions are met before finding the direction of move
            if f!="no":
                if gs.currentStateValues[m][n-1]==0:
                    direction.append("left")
                    pm=PossibleMove()
                    pm.dirc="left"
                    pm.xcord=m
                    pm.ycord=n
                    pm.shp=shape
                    pm.gs=self.move(m,n,shape,gs,"left")
                    possmovers.append(pm)
            f=self.boundryCheck(m,n+2,gs)
            if f!="no":    
                if gs.currentStateValues[m][n+2]==0:
                    direction.append("right")
                    pm=PossibleMove()
                    pm.dirc="right"
                    pm.xcord=m
                    pm.ycord=n
                    pm.shp=shape
                    pm.gs=self.move(m,n,shape,gs,"right")
                    possmovers.append(pm)
            f=self.boundryCheck(m-1,n+1,gs)
            if f!="no":    
                if gs.currentStateValues[m-1][n]==0 and gs.currentStateValues[m-1][n+1]==0:
                    direction.append("up")
                    pm=PossibleMove()
                    pm.dirc="up"
                    pm.xcord=m
                    pm.ycord=n
                    pm.shp=shape
                    pm.gs=self.move(m,n,shape,gs,"up")
                    possmovers.append(pm)
            f=self.boundryCheck(m+1,n+1,gs)
            if f!="no":    
                if gs.currentStateValues[m+1][n]==0 and gs.currentStateValues[m+1][n+1]==0:
                    direction.append("down")
                    pm=PossibleMove()
                    pm.dirc="down"
                    pm.xcord=m
                    pm.ycord=n
                    pm.shp=shape
                    pm.gs=self.move(m,n,shape,gs,"down")
                    possmovers.append(pm)
        if shape=="long":
            f=self.boundryCheck(m+1,n-1,gs)
            if f!="no":
                if gs.currentStateValues[m][n-1]==0 and gs.currentStateValues[m+1][n-1]:
                    direction.append("left")
                    pm=PossibleMove()
                    pm.dirc="left"
                    pm.xcord=m
                    pm.ycord=n
                    pm.shp=shape
                    pm.gs=self.move(m,n,shape,gs,"left")
                    possmovers.append(pm)
            
            f=self.boundryCheck(m+1,n+1,gs)
            if f!="no":    
                if gs.currentStateValues[m][n+1]==0 and gs.currentStateValues[m+1][n+1]:
                    direction.append("right")
                    pm=PossibleMove()
                    pm.dirc="right"
                    pm.xcord=m
                    pm.ycord=n
                    pm.shp=shape
                    pm.gs=self.move(m,n,shape,gs,"right")
                    possmovers.append(pm)
            f=self.boundryCheck(m-1,n,gs)
            if f!="no":    
                if gs.currentStateValues[m-1][n]==0:
                    direction.append("up")
                    pm=PossibleMove()
                    pm.dirc="up"
                    pm.xcord=m
                    pm.ycord=n
                    pm.shp=shape
                    pm.gs=self.move(m,n,shape,gs,"up")
                    possmovers.append(pm)
            f=self.boundryCheck(m+1,n,gs)
            if f!="no":    
                if gs.currentStateValues[m+1][n]==0:
                    direction.append("down") 
                    pm=PossibleMove()
                    pm.dirc="down"
                    pm.xcord=m
                    pm.ycord=n
                    pm.shp=shape
                    pm.gs=self.move(m,n,shape,gs,"down")
                    possmovers.append(pm)
                
        if shape=="smallsquare":
            f=self.boundryCheck(m,n-1,gs)
            if f!="no":
                if gs.currentStateValues[m][n-1]==0:
                    direction.append("left")
                    pm=PossibleMove()
                    pm.dirc="left"
                    pm.xcord=m
                    pm.ycord=n
                    pm.shp=shape
                    pm.gs=self.move(m,n,shape,gs,"left")
                    possmovers.append(pm)
            
            f=self.boundryCheck(m,n+1,gs)
            if f!="no":    
                if gs.currentStateValues[m][n+1]==0:
                    direction.append("right")
                    pm=PossibleMove()
                    pm.dirc="right"
                    pm.xcord=m
                    pm.ycord=n
                    pm.shp=shape
                    pm.gs=self.move(m,n,shape,gs,"right")
                    possmovers.append(pm)
            f=self.boundryCheck(m-1,n,gs)
            if f!="no":    
                if gs.currentStateValues[m-1][n]==0:
                    direction.append("up")
                    pm=PossibleMove()
                    pm.dirc="up"
                    pm.xcord=m
                    pm.ycord=n
                    pm.shp=shape
                    pm.gs=self.move(m,n,shape,gs,"up")
                    possmovers.append(pm)
            f=self.boundryCheck(m+1,n,gs)
            if f!="no":    
                if gs.currentStateValues[m+1][n]==0:
                    direction.append("down")
                    pm=PossibleMove()
                    pm.dirc="down"
                    pm.xcord=m
                    pm.ycord=n
                    pm.gs=self.move(m,n,shape,gs,"down")
                    pm.shp=shape
                    possmovers.append(pm)
        return possmovers  
    
#checking if block is at the boundry  

    def boundryCheck(self,b1,b2,gs):
        if gs.currentStateValues[b1][b2]==1:
            return "no"

#moving each block as per the direction passed as parameter . Shape of block determines how to move block       
    def move(self,m,n,shape,oldgs,direction):
        newgs = GameState(oldgs)
        if direction =="left":
            newgs.currentStateValues[m][n-1]=newgs.currentStateValues[m][n]
            newgs.currentStateValues[m][n]=0
            if shape =="wide":
                newgs.currentStateValues[m][n]=newgs.currentStateValues[m][n+1]
                newgs.currentStateValues[m][n+1]=0
            elif shape =="long":
                newgs.currentStateValues[m+1][n-1]=newgs.currentStateValues[m+1][n]
                newgs.currentStateValues[m+1][n]=0
                
        if direction =="right":
            newgs.currentStateValues[m][n+1]=newgs.currentStateValues[m][n]
            newgs.currentStateValues[m][n]=0
            if shape =="wide":
                newgs.currentStateValues[m][n+2]=newgs.currentStateValues[m][n+1]
            elif shape =="long":
                newgs.currentStateValues[m+1][n+1]=newgs.currentStateValues[m+1][n]
                newgs.currentStateValues[m+1][n]=0
                
        if direction =="up":
            newgs.currentStateValues[m-1][n]=newgs.currentStateValues[m][n]
            newgs.currentStateValues[m][n]=0
            if shape =="wide":
                newgs.currentStateValues[m-1][n+1]=newgs.currentStateValues[m][n+1]
                newgs.currentStateValues[m][n+1]=0
            elif shape =="long":
                newgs.currentStateValues[m][n]=newgs.currentStateValues[m+1][n]
                newgs.currentStateValues[m+1][n]=0         
                
        if direction =="down":
            newgs.currentStateValues[m+1][n]=newgs.currentStateValues[m][n]
            newgs.currentStateValues[m][n]=0
            if shape =="wide":
                newgs.currentStateValues[m+1][n+1]=newgs.currentStateValues[m][n+1]
                newgs.currentStateValues[m][n+1]=0
            elif shape =="long":
                newgs.currentStateValues[m+2][n]=newgs.currentStateValues[m+1][n]
        print("Updated state after moving", m, n, "to", direction)
        #newgs.printCurrentState()
        return newgs
#generating all the moves, selecting one at random,executing it,normalizing the resulting game state,
#if we have reached the goal, or if we have executed N moves, stop; otherwise, go back and select a move again    
    def randomWalks(self,gs,n):
        if n==0:
            return 
        
        allrandommoves=self.selectBlock(gs)
        #randomly choosing a move
        randomSelectedValue=random.choice(allrandommoves)
        print("randomly generated")
        randomSelectedValue.gs.printCurrentState()      
        print(randomSelectedValue.xcord,randomSelectedValue.ycord,randomSelectedValue.dirc)
        randomSelectedValue.gs.normalization()
        #checking if goalstate has been reached
        x=randomSelectedValue.gs.checkresult()
        if(x==0):
            print("gamestate reached via random walk")
            
        else:
            #go back and select a move again
            self.randomWalks(randomSelectedValue.gs,n-1)


from queue import Queue
import timeit
import random

rootMove = PossibleMove()
rootMove.gs = GameState("SBP-level1.txt")
#using the concept of queue to perform BFS and printing the dequed move till we reach the goal
mv = MoveBlock()
print ("\n \n ........Generating Random Walk........ \n\n")
mv.randomWalks(rootMove.gs,3)
print("\n \n .........Generating moves for BFS.......\n \n")

def BFSsearch():
    
    q = Queue()
    q.put(rootMove)
    nodesbfs=0
    result = rootMove.gs.checkresult()
    
    while result != 0 and not q.empty():
        dequeuedMove = q.get()
        dequeuedMove.gs.printCurrentState()
        #derive next possible moves from current gamestate
        for move in mv.selectBlock(dequeuedMove.gs):
            #check if gamestate derived from this move already exists
            if rootMove.checkIfGameStateAlreadyExists(move.gs):
                print("Move", move.xcord, move.ycord, move.dirc, "already exists. Skipping..")
            else:
                nodesbfs=nodesbfs+1
                dequeuedMove.nextPossibleMoves.append(move)
                result = move.gs.checkresult()
                q.put(move)
                if result == 0:
                    print("no. of nodes",nodesbfs)
                    break
print("BFS search time",timeit.timeit(BFSsearch,number=1))
#A function that solves a given sliding bricks puzzle using a depth-first strategy.
class Dfs:
    print("\n \n ........Generating moves for DFS........ \n\n")
    def __init__(self):
        self.result=[]
        self.nodesDFS=0
    #using recursive function to perform DFS
    def recDFS(self,mv):
        #check if goal has been reached
        if mv.gs.checkresult() == 0:
            return 1
        #iterating next possible moves from current node
        for move in mv.nextPossibleMoves:
            self.nodesDFS=self.nodesDFS+1
            if self.recDFS(move):
                self.result.append(move)
                return 1
        return 0
    
    def dfscheck(self):
        if self.recDFS(rootMove):
            print("Follow these steps to reach the goal")
            while len(self.result) > 0:
                #popping the nodes from stack
                move = self.result.pop()
                print("Move (", move.xcord, ",", move.ycord, ") to", move.dirc)
        else:
            print("Cannot reach goal")
        print("no. of nodes visited",self.nodesDFS) 
        
dfs1=Dfs()        
print("time for DFS",timeit.timeit(dfs1.dfscheck,number=1))

#A function that solves a given sliding bricks puzzle using an iterative deepening search strategy
class IDSImpl:

    print("\n\n.........Generating moves for IDS............\n\n")
    noofnodes=0
    result = []
    
    def __init__(self):
        self.noofnodes=0
        self.result=[]
   #performing DFS iteratively  
    def iDS(self, mv, lim):
        if lim==0:
            return 0
        if mv.gs.checkresult() == 0:
            return 1
        #iterating next possible moves from current node
        for move in mv.nextPossibleMoves:
            self.noofnodes=self.noofnodes+1
            if self.iDS(move,lim-1):
                self.result.append(move)
                return 1
        return 0
 #calling DFS for each level   
def calcIDS():
    ids = IDSImpl()
    for i in range(15):
        if ids.iDS(rootMove,i):
            print("visited nodes:", ids.noofnodes, "! Follow these steps to reach the goal")
            while len(ids.result) > 0:
                #popping the state from stack
                move = ids.result.pop()
                print("Move (", move.xcord, ",", move.ycord, ")", move.dirc)
            break
        else:
            print("keep moving", i)
print("ids time",timeit.timeit(calcIDS,number=1))


 
 ........Generating Random Walk........ 


Updated state after moving 1 1 to down
Updated state after moving 2 2 to left
Updated state after moving 3 2 to left
randomly generated
[1, 1, 1, 1, 1]
[1, 3, 2, 2, 1]
[1, 0, 4, 5, 1]
[-1, 6, 0, 7, 1]
[1, 1, 1, 1, 1]
3 2 left
Updated state after moving 1 1 to down
Updated state after moving 2 2 to left
Updated state after moving 2 2 to down
Updated state after moving 3 1 to right
Updated state after moving 3 1 to up
Updated state after moving 3 3 to left
randomly generated
[1, 1, 1, 1, 1]
[1, 3, 2, 2, 1]
[1, 0, 4, 5, 1]
[-1, 0, 6, 7, 1]
[1, 1, 1, 1, 1]
3 1 right
Updated state after moving 1 1 to down
Updated state after moving 2 2 to left
Updated state after moving 3 2 to left
randomly generated
[1, 1, 1, 1, 1]
[1, 3, 2, 2, 1]
[1, 4, 0, 5, 1]
[-1, 0, 6, 7, 1]
[1, 1, 1, 1, 1]
2 2 left

 
 .........Generating moves for BFS.......
 

[1, 1, 1, 1, 1]
[1, 3, 2, 2, 1]
[1, 0, 4, 5, 1]
[-1, 0, 6, 7, 1]
[1, 1, 1, 1, 1]
Updated state after moving 1 1

[1, 1, 1, 1, 1]
[1, 3, 2, 2, 1]
[1, 6, 6, 6, 1]
[-1, 0, 7, 0, 1]
[1, 1, 1, 1, 1]
Updated state after moving 3 2 to left
Updated state after moving 3 2 to right
Move 3 2 right already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 3, 2, 2, 1]
[1, 0, 4, 0, 1]
[-1, 7, 7, 7, 1]
[1, 1, 1, 1, 1]
Updated state after moving 1 1 to down
Updated state after moving 2 2 to left
Updated state after moving 2 2 to right
Move 1 1 down already exists. Skipping..
Move 2 2 left already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 2, 2, 0, 1]
[1, 5, 5, 0, 1]
[-1, 5, 6, 7, 1]
[1, 1, 1, 1, 1]
Updated state after moving 1 1 to right
Updated state after moving 2 1 to right
Updated state after moving 3 3 to up
Move 1 1 right already exists. Skipping..
Move 2 1 right already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 2, 2, 0, 1]
[1, 5, 5, 5, 1]
[-1, 0, 6, 7, 1]
[1, 1, 1, 1, 1]
Updated state after moving 1 1 to right
Updated state after moving 3 2 to left
Move 1 1 right already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 2, 2, 5, 1]
[

Move 2 1 up already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 3, 2, 2, 1]
[1, 6, 6, 0, 1]
[-1, 6, 0, 7, 1]
[1, 1, 1, 1, 1]
Updated state after moving 2 1 to right
Updated state after moving 3 3 to left
Updated state after moving 3 3 to up
Move 2 1 right already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 3, 2, 2, 1]
[1, 0, 6, 6, 1]
[-1, 0, 6, 7, 1]
[1, 1, 1, 1, 1]
Updated state after moving 1 1 to down
Updated state after moving 2 2 to left
Move 1 1 down already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 3, 2, 2, 1]
[1, 0, 6, 6, 1]
[-1, 6, 7, 0, 1]
[1, 1, 1, 1, 1]
Updated state after moving 1 1 to down
Updated state after moving 2 2 to left
Updated state after moving 3 1 to up
Updated state after moving 3 2 to right
Move 1 1 down already exists. Skipping..
Move 2 2 left already exists. Skipping..
Move 3 1 up already exists. Skipping..
Move 3 2 right already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 3, 2, 2, 1]
[1, 4, 6, 6, 1]
[-1, 7, 0, 0, 1]
[1, 1, 1, 1, 1]
Updated state after moving 2 2 to down
Upd

[1, 1, 1, 1, 1]
[1, 3, 2, 2, 1]
[1, 5, 5, 0, 1]
[-1, 6, 0, 7, 1]
[1, 1, 1, 1, 1]
Updated state after moving 2 1 to right
Updated state after moving 3 1 to right
Updated state after moving 3 3 to left
Updated state after moving 3 3 to up
Move 2 1 right already exists. Skipping..
Move 3 1 right already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 3, 2, 2, 1]
[1, 5, 5, 7, 1]
[-1, 0, 7, 0, 1]
[1, 1, 1, 1, 1]
Updated state after moving 2 3 to down
Updated state after moving 3 2 to left
Updated state after moving 3 2 to right
Move 3 2 left already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 3, 2, 2, 1]
[1, 7, 7, 0, 1]
[-1, 7, 7, 0, 1]
[1, 1, 1, 1, 1]
Updated state after moving 2 1 to right
Updated state after moving 3 1 to right
Move 2 1 right already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 3, 2, 2, 1]
[1, 0, 7, 7, 1]
[-1, 0, 7, 7, 1]
[1, 1, 1, 1, 1]
Updated state after moving 1 1 to down
Updated state after moving 2 2 to left
Updated state after moving 3 2 to left
Move 1 1 down already exists. Skipping

[1, 6, 6, 0, 1]
[-1, 0, 6, 7, 1]
[1, 1, 1, 1, 1]
Updated state after moving 2 1 to right
Updated state after moving 2 2 to right
Updated state after moving 3 3 to up
Move 2 1 right already exists. Skipping..
Move 2 2 right already exists. Skipping..
Move 3 3 up already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 3, 2, 2, 1]
[1, 4, 0, 0, 1]
[-1, 7, 6, 6, 1]
[1, 1, 1, 1, 1]
Updated state after moving 1 2 to down
Updated state after moving 2 1 to right
Updated state after moving 3 2 to up
Move 3 2 up already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 3, 0, 0, 1]
[1, 7, 2, 2, 1]
[-1, 7, 0, 7, 1]
[1, 1, 1, 1, 1]
Updated state after moving 1 1 to right
Updated state after moving 2 2 to up
Updated state after moving 3 3 to left
Move 2 2 up already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 3, 2, 2, 1]
[1, 7, 0, 0, 1]
[-1, 7, 7, 0, 1]
[1, 1, 1, 1, 1]
Updated state after moving 1 2 to down
Updated state after moving 2 1 to right
Updated state after moving 3 1 to right
Move 1 2 down already exists. Skipping.

...

Move 3 1 up already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 0, 2, 2, 1]
[1, 0, 3, 6, 1]
[-1, 0, 6, 6, 1]
[1, 1, 1, 1, 1]
Updated state after moving 1 2 to left
Updated state after moving 2 2 to left
Updated state after moving 3 2 to left
Move 1 2 left already exists. Skipping..
Move 2 2 left already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 0, 2, 2, 1]
[1, 6, 6, 6, 1]
[-1, 0, 0, 6, 1]
[1, 1, 1, 1, 1]
Updated state after moving 1 2 to left
Updated state after moving 2 1 to down
Move 1 2 left already exists. Skipping..
Move 2 1 down already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 0, 2, 2, 1]
[1, 3, 6, 0, 1]
[-1, 0, 6, 0, 1]
[1, 1, 1, 1, 1]
Updated state after moving 1 2 to left
Updated state after moving 2 1 to up
Updated state after moving 2 1 to down
Move 1 2 left already exists. Skipping..
Move 2 1 up already exists. Skipping..
Move 2 1 down already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 0, 2, 2, 1]
[1, 3, 0, 6, 1]
[-1, 6, 6, 0, 1]
[1, 1, 1, 1, 1]
Updated state after moving 1 2 to left
Upd

[1, 1, 1, 1, 1]
[1, 2, 2, 0, 1]
[1, 3, 5, 0, 1]
[-1, 0, 5, 7, 1]
[1, 1, 1, 1, 1]
Updated state after moving 1 1 to right
Updated state after moving 2 1 to down
Updated state after moving 2 2 to right
Updated state after moving 3 3 to up
Move 1 1 right already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 0, 2, 2, 1]
[1, 0, 5, 0, 1]
[-1, 5, 5, 7, 1]
[1, 1, 1, 1, 1]
Updated state after moving 1 2 to left
Updated state after moving 2 2 to left
Updated state after moving 2 2 to right
Updated state after moving 3 3 to up
Move 1 2 left already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 0, 2, 2, 1]
[1, 3, 0, 5, 1]
[-1, 0, 0, 5, 1]
[1, 1, 1, 1, 1]
Updated state after moving 1 2 to left
Updated state after moving 2 1 to right
Updated state after moving 2 1 to up
Updated state after moving 2 1 to down
Move 1 2 left already exists. Skipping..
Move 2 1 up already exists. Skipping..
Move 2 1 down already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 0, 2, 2, 1]
[1, 3, 7, 7, 1]
[-1, 0, 7, 0, 1]
[1, 1, 1, 1, 1]
Update

Move 1 1 right already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 0, 2, 2, 1]
[1, 3, 0, 4, 1]
[-1, 6, 6, 7, 1]
[1, 1, 1, 1, 1]
Updated state after moving 1 2 to left
Updated state after moving 2 1 to right
Updated state after moving 2 1 to up
Updated state after moving 2 3 to left
Move 1 2 left already exists. Skipping..
Move 2 1 up already exists. Skipping..
Move 2 3 left already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 0, 2, 2, 1]
[1, 3, 4, 7, 1]
[-1, 6, 6, 0, 1]
[1, 1, 1, 1, 1]
Updated state after moving 1 2 to left
Updated state after moving 2 1 to up
Updated state after moving 2 3 to down
Updated state after moving 3 1 to right
Move 1 2 left already exists. Skipping..
Move 2 1 up already exists. Skipping..
Move 2 3 down already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 3, 2, 2, 1]
[1, 0, 4, 7, 1]
[-1, 0, 6, 6, 1]
[1, 1, 1, 1, 1]
Updated state after moving 1 1 to down
Updated state after moving 2 2 to left
Updated state after moving 3 2 to left
Move 1 1 down already exists. Skipping..
Move 

Move 1 1 down already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 3, 2, 2, 1]
[1, 0, 0, 0, 1]
[-1, 6, 6, 6, 1]
[1, 1, 1, 1, 1]
Updated state after moving 1 1 to down
Updated state after moving 1 2 to down
Updated state after moving 3 1 to up
Updated state after moving 3 2 to up
Move 1 1 down already exists. Skipping..
Move 3 1 up already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 3, 2, 2, 1]
[1, 6, 6, 6, 1]
[-1, 0, 0, 0, 1]
[1, 1, 1, 1, 1]
Updated state after moving 2 1 to down
Updated state after moving 2 2 to down
Move 2 1 down already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 0, 2, 2, 1]
[1, 3, 6, 6, 1]
[-1, 0, 0, 6, 1]
[1, 1, 1, 1, 1]
Updated state after moving 1 2 to left
Updated state after moving 2 1 to up
Updated state after moving 2 1 to down
Move 2 1 up already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 3, 2, 2, 1]
[1, 6, 6, 0, 1]
[-1, 0, 0, 6, 1]
[1, 1, 1, 1, 1]
Updated state after moving 2 1 to right
Updated state after moving 2 1 to down
Updated state after moving 3 3 to left
Updated stat

[1, 1, 1, 1, 1]
[1, 2, 2, 7, 1]
[1, 7, 6, 0, 1]
[-1, 0, 6, 6, 1]
[1, 1, 1, 1, 1]
Updated state after moving 1 3 to down
Updated state after moving 2 1 to down
Updated state after moving 2 2 to right
Updated state after moving 3 2 to left
Move 1 3 down already exists. Skipping..
Move 3 2 left already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 2, 2, 0, 1]
[1, 0, 6, 7, 1]
[-1, 6, 6, 6, 1]
[1, 1, 1, 1, 1]
Updated state after moving 1 1 to right
Updated state after moving 2 2 to left
Updated state after moving 2 3 to up
Move 1 1 right already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 0, 2, 2, 1]
[1, 0, 7, 7, 1]
[-1, 0, 7, 7, 1]
[1, 1, 1, 1, 1]
Updated state after moving 1 2 to left
Updated state after moving 2 2 to left
Updated state after moving 3 2 to left
Move 1 2 left already exists. Skipping..
Move 3 2 left already exists. Skipping..
[1, 1, 1, 1, 1]
[1, 6, 2, 2, 1]
[1, 6, 0, 6, 1]
[-1, 0, 6, 7, 1]
[1, 1, 1, 1, 1]
Updated state after moving 2 3 to left
Updated state after moving 3 2 to left
Up